## Установка CLIP

In [11]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\denis\appdata\local\temp\pip-req-build-67vmoiok
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369471 sha256=b1a222253a77172c99958cfe76f6b7e0ad9ed2b9a966ff885198d7f72670bdfb
  Stored in directory: C:\Users\Denis\AppData\Local\Temp\pip-ephem-wheel-cache-mc08a_x7\wheels\41\36\e0\af6191bd08c254b17d779b05de8282c9ea81ef7048abbe290b
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41914 sha256=461af45b30bf51f712b89bfc69ca4d09db1ed7853b6f0a7ade39858514ca650c
  Stored in directory: c:\users\denis\appdata\local\pip\cache\wheels\ff\2a\24\75041425faf3347ab146a4a3d0484f723b2c44a7966a06e3f0
Successfully built clip ftfy


  Running command git clone -q https://github.com/openai/CLIP.git 'C:\Users\Denis\AppData\Local\Temp\pip-req-build-67vmoiok'
You should consider upgrading via the 'd:\captcha\synvae\scripts\python.exe -m pip install --upgrade pip' command.


## Импорт библиотек

In [ ]:
import os
import json
from tqdm.auto import tqdm
import time
import gzip
from PIL import Image
import numpy as np
import pickle
from IPython.display import display_png

import torch
import clip

## Определение директорий с артефактами парсинга

In [3]:
FOLDERS = ['desktop_chunks20k']

## Загрузка модели CLIP

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device, download_root='CLIP')
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)


## Получение эмбеддингов CLIP

In [14]:
processed_urls = []
preds = []
total_pbar = tqdm()
batch = []
savefile = 'clipVIT_scores_20k.pickle.gz'
for folder in FOLDERS:
    for cnt, file in tqdm(enumerate(map(lambda x: os.path.join(folder, x), os.listdir(folder)))):
        try:
            with gzip.open(file, 'rb') as f:
                chunk = pickle.load(f)
        except:
            continue
        if len(chunk['labels']):
            processed_urls.extend(list(chunk['labels'][:, 0]))
            for img in chunk['data']:
                with torch.no_grad():
                    preds.append(
                        model.encode_image(
                            preprocess(
                                Image.fromarray(img).convert('RGB')
                            ).unsqueeze(0).to(device)
                        ).detach().cpu().numpy().flatten()
                    )
                total_pbar.update(1)
        if cnt % 1000 == 0:
            with gzip.open(savefile, 'wb') as f:
                pickle.dump(dict(domains=processed_urls, scores=preds), f, protocol=-1)
with gzip.open(savefile, 'wb') as f:
    pickle.dump(dict(domains=processed_urls, scores=preds), f, protocol=-1)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Получение тайтлов ссылок

In [20]:
processed_urls = []
total_pbar = tqdm()
for folder in FOLDERS:
    for cnt, file in tqdm(enumerate(map(lambda x: os.path.join(folder, x), os.listdir(folder)))):
        try:
            with gzip.open(file, 'rb') as f:
                chunk = pickle.load(f)
        except:
            continue
        if len(chunk['labels']):
            for (domain, title), html in zip(chunk['labels'], chunk['html']):
                processed_urls.append((domain, title, html))
                total_pbar.update(1)
with gzip.open('domain20k_text.pickle.gz', 'wb') as f:
    pickle.dump(dict(domain=np.array([x[0] for x in processed_urls]),
                     title=np.array([x[1] for x in processed_urls])), f, protocol=-1)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Получение HTML ссылок

In [31]:
with gzip.open('domain20k_html.txt.gz', 'wt', encoding='utf-8') as f:
    for x in tqdm(processed_urls):
        if '\n=\n=\n' in x[2]:
            print('!!!', x[0], x[1])
            break
        f.write(x[2]+'\n=\n=\n')

  0%|          | 0/16334 [00:00<?, ?it/s]